In [136]:
# Include Libraries
import numpy as np
import tensorflow as tf
import gym
import tensorflow.contrib.layers as layers
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import policies
import value_functions as vfuncs
import utils_pg as utils

# Environment setup
#env = "CartPole-v0"
env="InvertedPendulum-v2"
#env = "Hopper-v2"

# discrete = isinstance(env.action_space, gym.spaces.Discrete)
# observation_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n if discrete else env.action_space.shape[0]
max_ep_len = 1000
num_traj = 10
#traj_length = max_ep_len*(observation_dim + 2)
latent_size = 10
use_baseline = True



In [137]:
# Feed forward network (multi-layer-perceptron, or mlp)

def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):
    '''
    Build a feed forward network
    '''
    Input = mlp_input
    with tf.variable_scope(scope):
        # Dense Layers
        for i in range(n_layers-1):
            dense = tf.layers.dense(inputs = Input, units = size, activation = tf.nn.relu, bias_initializer=tf.constant_initializer(1.0))
            Input = dense
        # Fully Connected Layer
        out = layers.fully_connected(inputs = Input, num_outputs = output_size, activation_fn=output_activation)
    return out

In [138]:
class MetaLearner():
    def __init__(self, env, max_ep_len, num_traj,latent_size ):
        self.env = gym.make(env)
        self.discrete = isinstance(self.env.action_space, gym.spaces.Discrete)
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n if self.discrete else self.env.action_space.shape[0]
        self.max_ep_len = max_ep_len
        self.num_traj = num_traj
        self.traj_length = self.max_ep_len*(self.observation_dim + 2) # TO Change
        self.use_baseline = True
        self.latent_size = latent_size
        self.feature_size = self.observation_dim + 1 + self.env.action_space.shape[0] # HC
        self.lr = 3e-3
        self.num_layers = 2
        self.layers_size = 32
        self.num_mdps = 10
        self.model = self.env.env.model
        self.sim = MjSim(self.model)
        self.gamma = 0.95
        self.decoder_len = 32

        # build model
        self.ConstructGraph()
    
    def add_placeholders(self):
        self.observation_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        if(self.discrete):
            self.action_placeholder_explore = tf.placeholder(tf.int32, shape=(None))
            self.action_placeholder_exploit = tf.placeholder(tf.int32, shape=(None))
        else:
            self.action_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.action_dim))
            self.action_placeholder_exploit= tf.placeholder(tf.float32, shape=(None,self.action_dim))

        self.baseline_target_placeholder = tf.placeholder(tf.float32, shape= None)
        self.advantage_placeholder_explore = tf.placeholder(tf.float32, shape=(None))
        
        #self.encoder_input_placeholder = tf.placeholder(tf.float32, shape= (self.num_traj,self.traj_length))
        self.encoder_input_placeholder = tf.placeholder(tf.float32, [None, None, self.feature_size])
        self.decoder_input_placeholder = tf.placeholder(tf.float32, shape= (1,self.latent_size))
        self.sequence_length_placeholder = tf.placeholder(tf.int32, [None, ])
        
        self.observation_placeholder_exploit = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        #TODO
        self.advantage_placeholder_exploit = tf.placeholder(tf.float32, shape=(None))
        
        
    def build_policy_explore(self, scope = "policy_explore"):
        if (self.discrete):
            self.action_logits = build_mlp(self.observation_placeholder_explore,self.action_dim,scope = scope,n_layers=self.num_layers,size = self.layers_size,output_activation=None)
            self.explore_action = tf.multinomial(self.action_logits,1)
            self.explore_action = tf.squeeze(self.explore_action, axis=1)
            self.explore_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.action_logits, labels = self.action_placeholder_explore)

        else:   
            action_means = build_mlp(self.observation_placeholder_explore,self.action_dim,scope,n_layers=self.num_layers, size = self.layers_size,output_activation=None)
            init = tf.constant(np.random.rand(1, 2))
            #log_std = tf.get_variable("log_std", [self.action_dim])
            log_std = tf.get_variable(
                'log_std',
                shape=[1, self.action_dim],
                dtype=tf.float32,
                initializer=tf.zeros_initializer(),
                trainable=True
            )
            log_std = tf.log(tf.exp(log_std) + 1.)
            log_std = tf.tile(log_std,
                                   [tf.shape(action_means)[0], 1])
            self.explore_action = tf.random_normal((1,),
                                                   action_means,
                                                   log_std)
            dist = tf.contrib.distributions.MultivariateNormalDiag(action_means, log_std)
            self.explore_logprob = dist.prob(self.action_placeholder_explore, name = 'log_prob')
            #self.explore_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            #mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
            #self.explore_logprob =  mvn.log_prob(value = self.action_placeholder_explore, name='log_prob')

    
    
    def build_policy_exploit(self, scope = "policy_exploit"):
        if(self.discrete):
            #self.exploit_action_logits = (tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W1) + self.d_B1), self.d_W2) + self.d_B2),self.d_W3) + self.d_B3)
            #self.exploit_action_logits = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            self.exploit_action_logits = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            self.exploit_action = tf.multinomial(self.exploit_action_logits,1)
            self.exploit_action = tf.squeeze(self.exploit_action, axis=1)
            self.exploit_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.exploit_action_logits, labels = self.action_placeholder_exploit)
        else:
            action_means = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            init = tf.constant(np.random.rand(1, 2))
            log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
            self.exploit_action = action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            self.exploit_logprob = utils.gauss_log_prob(mu=action_means, logstd=log_std, x=self.action_placeholder_exploit)
            
#             init = tf.constant(np.random.rand(1, 2))
#             log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
#             self.exploit_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
#             mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
#             self.exploit_logprob =  mvn.log_prob(value = self.action_placeholder_exploit, name='exploit_log_prob')

        #self.loss_grads_exploit = self.exploit_logprob * self.advantage_placeholder_exploit
        
    def NNEncoder(self, scope = "NNEncoder"):
        self.Z = build_mlp(self.encoder_input_placeholder,self.latent_size,scope = scope,n_layers=3,size = 60,output_activation=None)
    
 

    # input [num_traj, length, features (obs + action + reward) ]
    def LSTMEncoder(self, scope = "LSTMEncoder"):
        self.hidden_size = 64
        initializer = tf.random_uniform_initializer(-1, 1)
        cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, self.feature_size, initializer=initializer)
        cell_out = tf.contrib.rnn.OutputProjectionWrapper(cell, self.latent_size)
        self.output, _ = tf.nn.dynamic_rnn(cell_out,self.encoder_input_placeholder,self.sequence_length_placeholder,dtype=tf.float32,)
        batch_size = tf.shape(self.output)[0]
        max_length = tf.shape(self.output)[1]
        out_size = int(self.output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (self.sequence_length_placeholder - 1)
        flat = tf.reshape(self.output, [-1, out_size])
        self.Z = tf.reduce_mean(tf.gather(flat, index), axis = 0)
        
    
    def Decoder(self, decoder_out_dim, scope):
        return build_mlp(self.decoder_input_placeholder,decoder_out_dim,scope = scope,n_layers=1,size = 64,output_activation=None)
    
    
    def sample_paths_explore(self, env,mdp_num,Test = False, num_episodes = None):
        paths = []
        self.length = []
        episode_rewards = []
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        for i in range(self.num_traj):
            pad = False
            state = env.reset()
            new_states,states,new_actions, actions, rewards,new_rewards = [], [], [], [], [],[]
            episode_reward = 0
            for step in range(self.max_ep_len):
                if (pad):
                    states.append([0]*self.observation_dim)
                    if(self.discrete):
                        actions.append(0)
                    else:
                        actions.append([0]*self.action_dim)
                    rewards.append(0)
                else:
                    states.append(state)
                    new_states.append(state)
                    #action = self.sess.run(self.explore_action, feed_dict={self.observation_placeholder_explore : states[-1][None]})[0]
                    action = self.policyfn.sample_action(state) #NEW
                    state, reward, done, info = env.step(action)
                    actions.append(action)
                    new_actions.append(action)
                    rewards.append(reward)
                    new_rewards.append(reward)
                    episode_reward += reward
                    if (done or step == self.max_ep_len-1):
                        self.length.append(step + 1)
                        pad = True 
                        
            episode_rewards.append(episode_reward)            
            #print("explore",np.array(actions))
            path = {"new_obs" : np.array(new_states),"observation" : np.array(states),
                                "reward" : np.array(rewards),"new_acs" : np.array(new_actions),
                                "action" : np.array(actions),"new_rewards":np.array(new_rewards)}
            paths.append(path)
        return paths, episode_rewards
    
    def sample_paths_exploit(self, env,Z,mdp_num,Test = False, num_episodes = None):
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        
        paths = []
        num = 0
        episode_rewards = []
        for i in range(self.num_traj):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0
            for step in range(self.max_ep_len):
                states.append(state)
                action = self.sess.run(self.exploit_action, feed_dict={self.observation_placeholder_exploit : state[None], self.decoder_input_placeholder: Z})[0]
                state, reward, done, info = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
            #print("exploit",np.array(actions))
            path = {"observation" : np.array(states),
                                "reward" : np.array(rewards),
                                "action" : np.array(actions)}
            paths.append(path)
 
        #print("exploit success: ", num)
        return paths, episode_rewards
    
    def get_returns(self,paths, explore = False):
        all_returns = []
        m = 0
        for path in paths:
            rewards = path["reward"]
            returns = []
            if(explore):
                length = self.length[m]
            else:
                length = len(rewards)
            for i in range(length):
                path_returns = 0
                k = 0
                for j in range(i,length):
                    path_returns = path_returns + rewards[j]*(self.gamma)**k
                    k = k+1
                returns.append(path_returns)
            all_returns.append(returns)
            m+=1
        returns = np.concatenate(all_returns)
        return returns
    
    def stack_trajectories(self,paths):
        trajectories = []
        for path in paths:
            rewards = path["reward"]
            states = path["observation"]
            action = path["action"]
            
            SAR = []
            for i in range(len(states)):
                SAR.append(list(states[i]) + list(action[i]) + [rewards[i]])
            trajectories.append(SAR)
        return np.array(trajectories)
    
    def addBaseline(self):
        self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=self.num_layers, size = self.layers_size,output_activation=None)
        self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
        baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
        self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)

    def calculate_advantage(self,returns, observations):
        if (self.use_baseline):
            baseline = self.sess.run(self.baseline, {input_placeholder:observations})
            adv = returns - baseline
            adv = (adv - np.mean(adv))/np.std(adv)
        else:
            adv = returns
        return adv
    

        
    def ConstructGraph(self):
        tf.reset_default_graph()
        self.sess = tf.Session()
        
        self.vf = vfuncs.NnValueFunction(session=self.sess,ob_dim=self.observation_dim)
        self.policyfn = policies.GaussianPolicy(self.sess,self.observation_dim, self.action_dim)
        
        self.add_placeholders()
        
#         self.add_placeholders()
        
#         self.build_policy_explore()
#         self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
#         self.loss_grads_explore = -self.explore_logprob * self.advantage_placeholder_explore
#         self.tvars_explore = tf.trainable_variables()
#         self.gradients_explore = tf.gradients(self.explore_policy_loss,self.tvars_explore)
        
#         #self.addBaseline()
        
#         self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=1, size = 16,output_activation=None)
#         self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
#         baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
#         self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)
        
#         #Encoder LSTM
        self.LSTMEncoder()
        
        
        #decoder weights
        #self.d_W1 = self.Decoder(scope = "W1", decoder_out_dim = self.observation_dim*self.decoder_len)
        self.d_W2 = self.Decoder(scope = "W2", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*action_dim)
        self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*self.action_dim)
        
        #self.d_W1 = ((self.d_W1 - (tf.reduce_max(self.d_W1) + tf.reduce_min(self.d_W1))/2)/(tf.reduce_max(self.d_W1) - tf.reduce_min(self.d_W1)))*2 
        #self.d_W2 = ((self.d_W2 - (tf.reduce_max(self.d_W2) + tf.reduce_min(self.d_W2))/2)/(tf.reduce_max(self.d_W2) - tf.reduce_min(self.d_W2)))*2 
        #self.d_W3 = ((self.d_W3 - (tf.reduce_max(self.d_W3) + tf.reduce_min(self.d_W3))/2)/(tf.reduce_max(self.d_W3) - tf.reduce_min(self.d_W3)))*2 
        
        #self.d_W1 = tf.reshape(self.d_W1, [self.observation_dim, self.decoder_len])
        self.d_W2 = tf.reshape(self.d_W2, [self.observation_dim, self.decoder_len])
        #self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        
        # decoder output bias
        #self.d_B1 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B1"), [self.decoder_len])
        self.d_B2 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B2"), [self.decoder_len])
        self.d_B3 = tf.reshape(self.Decoder(decoder_out_dim = self.action_dim, scope = "B3"), [self.action_dim])
        #self.d_B1 = ((self.d_B1 - (tf.reduce_max(self.d_B1) + tf.reduce_min(self.d_B1))/2)/(tf.reduce_max(self.d_B1) - tf.reduce_min(self.d_B1)))*2 
        #self.d_B2 = ((self.d_B2 - (tf.reduce_max(self.d_B2) + tf.reduce_min(self.d_B2))/2)/(tf.reduce_max(self.d_B2) - tf.reduce_min(self.d_B2)))*2 
        
        
        # exploit policy
        self.build_policy_exploit()
        #self.d = [self.d_W1, self.d_B1, self.d_W2, self.d_B2, self.d_W3, self.d_B3]
        self.exploit_policy_loss = -tf.reduce_sum(self.exploit_logprob * self.advantage_placeholder_exploit)
        self.d = [self.d_W3, self.d_B3]
        self.gradients_exploit = tf.gradients(self.exploit_policy_loss,self.d)
        
        # train encoder and decoder
        adam_optimizer_exploit =  tf.train.AdamOptimizer(self.lr*0.3)
        self.output_train_op = adam_optimizer_exploit.minimize(self.exploit_policy_loss)
        # train original network
#         adam_optimizer_explore = tf.train.AdamOptimizer(self.lr)
#         self.input_train_op = adam_optimizer_explore.minimize(self.explore_policy_loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
    
    def initialize(self):
        self.ConstructGraph()
#         # create tf session
#         self.sess = tf.Session()
        
#         # initiliaze all variables
#         init = tf.global_variables_initializer()
#         self.sess.run(init)
    
    def train_step(self): 
        # sample num_traj*num_MDPs
        for i in range(self.num_mdps):
            explore_paths, explore_rewards = self.sample_paths_explore(self.env,10*np.random.rand())
            observations_explore = np.concatenate([path["observation"] for path in explore_paths])
            new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
            new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
            actions_explore = np.concatenate([path["action"] for path in explore_paths])
            rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
            
            vtargs, vpreds, advantages_explore = [], [], []
            for path in explore_paths:
                rew_t = path["new_rewards"]
                return_t = utils.discount(rew_t, self.gamma)
                vpred_t = self.vf.predict(path["new_obs"])
                adv_t = return_t - vpred_t
                advantages_explore.append(adv_t)
                vtargs.append(return_t)
                vpreds.append(vpred_t)
                
                
            #returns_explore = self.get_returns(explore_paths, explore = True)
            advantages_explore = np.concatenate(advantages_explore)
            advantages_explore = (advantages_explore - advantages_explore.mean()) / (advantages_explore.std() + 1e-8)
            vtarg_n = np.concatenate(vtargs)
            vpred_n = np.concatenate(vpreds)
            #update baseline
            #print(advantages_explore.shape,new_observations_explore.shape)
            self.vf.fit(new_observations_explore, vtarg_n)
            
            surr_loss, oldmean_na, oldlogstd_a = self.policyfn.update_policy(
                    new_observations_explore, new_actions_explore, advantages_explore, self.lr)
            
            print("average reward explore:", np.mean(explore_rewards))
            
            #print(returns_explore)
#             print("average reward explore: MDP", i, np.sum(explore_rewards)/num_traj, len(explore_rewards))

#             baseline_explore = self.sess.run(self.baseline, {self.observation_placeholder_explore:new_observations_explore})
#             adv = returns_explore - np.squeeze(baseline_explore)
#             advantages_explore = (adv - np.mean(adv))/(np.std(adv) + 1e-12)


#             # update the baseline

#             self.sess.run(self.update_baseline_op, {self.observation_placeholder_explore:new_observations_explore, 
#                                            self.baseline_target_placeholder : returns_explore})

            # calculate explore gradients
    #         grads_explore = self.sess.run(self.gradients_explore, feed_dict={
    #                     self.observation_placeholder_explore : observations_explore,
    #                     self.action_placeholder_explore : actions_explore,
    #                     self.advantage_placeholder_explore : returns_explore})
            #print("explore",grads_explore )
            # form trajectory matrix

            M = np.array(self.stack_trajectories(explore_paths))


            #encoder LSTM
            Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
            Z = np.reshape(Z,[1,len(Z)])
#             #print("Z",Z)
#             #print(self.sess.run(self.d, feed_dict = {self.decoder_input_placeholder: Z}))
#             #print("d",self.d)
#             # sample paths
#             tvars = tf.trainable_variables()
#             tvars_vals = self.sess.run(tvars[-5:-1])
#             #print(tvars_vals)
            for j in range(10):
                exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,10*np.random.rand())
                # get observations, actions and rewards
                observations_exploit = np.concatenate([path["observation"] for path in exploit_paths])
                actions_exploit = np.concatenate([path["action"] for path in exploit_paths])
                rewards_exploit = np.concatenate([path["reward"] for path in exploit_paths])
                returns_exploit = self.get_returns(exploit_paths)
                print("average reward exploit: MDP", i, np.sum(exploit_rewards) / num_traj, len(exploit_rewards))


#             # exploit grads
#     #         grads_exploit = self.sess.run(self.gradients_exploit,feed_dict={
#     #                     self.observation_placeholder_exploit : observations_exploit,
#     #                     self.action_placeholder_exploit : actions_exploit,
#     #                     self.advantage_placeholder_exploit : returns_exploit,
#     #                     self.decoder_input_placeholder: Z})

            #train encoder and decoder network
                self.sess.run(self.output_train_op, feed_dict={
                                self.observation_placeholder_exploit : observations_exploit,
                                self.action_placeholder_exploit : actions_exploit,
                                self.advantage_placeholder_exploit : returns_exploit,
                                self.decoder_input_placeholder: Z})

            # find advantage for input network
    #         advantage_explore = 0
    #         for i in range(len(grads_exploit)):
    #             l1 = grads_exploit[i]
    #             l2 = grads_explore[i]
    #             advantage_explore = advantage_explore + np.matmul(l1.flatten(), l2.flatten())

            # train input policy
#             self.sess.run(self.input_train_op, feed_dict={
#                             self.observation_placeholder_explore : new_observations_explore,
#                             self.action_placeholder_explore : new_actions_explore,
#                             self.advantage_placeholder_explore : advantages_explore})
    
    def test(self):
        explore_paths, explore_rewards = self.sample_paths_explore(self.env,20*np.random.rand())
        print("gravity: ",self.sim.model.opt.gravity )
        M = self.stack_trajectories(explore_paths)
        Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
        Z = np.reshape(Z,[1,len(Z)])
        # sample paths
        exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,20*np.random.rand())
        print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
        
    def train(self):
        self.initialize()
        num_epochs = 200
        for epoch in range(num_epochs):
            self.num_traj = 20
            print("epoch number: ", epoch)
            self.train_step()
            print("evaluating on new MDPs")
            self.num_traj = 400
            self.test()

            
            

In [139]:
a = MetaLearner(env, max_ep_len, num_traj, latent_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [140]:
a.train()

epoch number:  0
average reward explore: 7.433057022770922
average reward exploit: MDP 0 13.146825934476633 20
average reward exploit: MDP 0 37.918129129707715 20
average reward exploit: MDP 0 54.49108160179552 20
average reward exploit: MDP 0 66.36915720000206 20
average reward exploit: MDP 0 67.02055270531001 20
average reward exploit: MDP 0 77.99376205030164 20
average reward exploit: MDP 0 76.33559053099799 20
average reward exploit: MDP 0 76.20290403605632 20
average reward exploit: MDP 0 72.80569838767053 20
average reward exploit: MDP 0 83.09025679225665 20
average reward explore: 10.023697805549068
average reward exploit: MDP 1 35.174874416523984 20
average reward exploit: MDP 1 48.30036971181886 20
average reward exploit: MDP 1 63.37136103682455 20
average reward exploit: MDP 1 68.72944317678463 20
average reward exploit: MDP 1 64.58220323068957 20
average reward exploit: MDP 1 59.802738744713636 20
average reward exploit: MDP 1 66.75063745115361 20
average reward exploit: MDP

average reward exploit: MDP 4 37.91554004365383 20
average reward exploit: MDP 4 37.715788154869905 20
average reward exploit: MDP 4 37.820646269472604 20
average reward exploit: MDP 4 37.58602610761451 20
average reward exploit: MDP 4 37.49671001778866 20
average reward explore: 64.17152019505104
average reward exploit: MDP 5 37.645485733304554 20
average reward exploit: MDP 5 37.99572518308127 20
average reward exploit: MDP 5 37.591439671608335 20
average reward exploit: MDP 5 37.021231365880325 20
average reward exploit: MDP 5 37.27947072460943 20
average reward exploit: MDP 5 37.37662795296653 20
average reward exploit: MDP 5 37.97903007381281 20
average reward exploit: MDP 5 36.71246626474816 20
average reward exploit: MDP 5 37.10821975800601 20
average reward exploit: MDP 5 36.988151847088076 20
average reward explore: 69.45749829925246
average reward exploit: MDP 6 35.329494812075524 20
average reward exploit: MDP 6 34.657552096243755 20
average reward exploit: MDP 6 35.17051327

average reward exploit: MDP 9 20.23383545617598 20
average reward exploit: MDP 9 19.64334830450773 20
average reward exploit: MDP 9 17.02804363865916 20
average reward exploit: MDP 9 16.939879336205166 20
average reward exploit: MDP 9 16.318936640052762 20
average reward exploit: MDP 9 15.899041052240563 20
average reward exploit: MDP 9 14.667128640043842 20
average reward exploit: MDP 9 15.018694753192978 20
average reward exploit: MDP 9 17.068805135736483 20
average reward exploit: MDP 9 17.3951878496376 20
evaluating on new MDPs
gravity:  [  0.           0.         -24.39416799]
average reward exploit 211.6998216426823 400
epoch number:  3
average reward explore: 112.7451024771945
average reward exploit: MDP 0 18.562053713463197 20
average reward exploit: MDP 0 19.082665946633718 20
average reward exploit: MDP 0 18.699506109862877 20
average reward exploit: MDP 0 18.183753284597355 20
average reward exploit: MDP 0 18.959293833461096 20
average reward exploit: MDP 0 18.26636036219680

average reward exploit: MDP 3 80.93431584595687 20
average reward exploit: MDP 3 92.89986743201814 20
average reward exploit: MDP 3 99.60624486824688 20
average reward exploit: MDP 3 88.5217702360836 20
average reward exploit: MDP 3 96.04849541178245 20
average reward exploit: MDP 3 94.91055003952154 20
average reward exploit: MDP 3 113.23605585941645 20
average reward explore: 153.5405946075899
average reward exploit: MDP 4 96.63746993672339 20
average reward exploit: MDP 4 74.76682710460129 20
average reward exploit: MDP 4 34.62500942589418 20
average reward exploit: MDP 4 14.8043309941804 20
average reward exploit: MDP 4 13.110294125128817 20
average reward exploit: MDP 4 15.592147836972671 20
average reward exploit: MDP 4 7.5305573960200975 20
average reward exploit: MDP 4 11.164417031091023 20
average reward exploit: MDP 4 11.127456254661942 20
average reward exploit: MDP 4 9.469068954105817 20
average reward explore: 168.28501487964817
average reward exploit: MDP 5 11.38027039480

average reward exploit: MDP 7 63.39326254962327 20
average reward explore: 163.05340067943007
average reward exploit: MDP 8 64.9038545157216 20
average reward exploit: MDP 8 70.18926469979716 20
average reward exploit: MDP 8 65.61775274956702 20
average reward exploit: MDP 8 74.52719346971966 20
average reward exploit: MDP 8 67.97620484065095 20
average reward exploit: MDP 8 69.37637511096732 20
average reward exploit: MDP 8 73.28756493026331 20
average reward exploit: MDP 8 68.96333998834203 20
average reward exploit: MDP 8 73.63853272191345 20
average reward exploit: MDP 8 75.50361112040578 20
average reward explore: 158.9359764327874
average reward exploit: MDP 9 75.02561677245143 20
average reward exploit: MDP 9 80.42993609178875 20
average reward exploit: MDP 9 76.6863869952492 20
average reward exploit: MDP 9 80.0377971701111 20
average reward exploit: MDP 9 86.14947624059423 20
average reward exploit: MDP 9 77.16424439279231 20
average reward exploit: MDP 9 79.54499151937912 20


...

average reward exploit: MDP 8 28.282272585507133 20
average reward explore: 605.7543762922581
average reward exploit: MDP 9 30.987915069218616 20
average reward exploit: MDP 9 32.4212723080586 20
average reward exploit: MDP 9 31.915753346622722 20
average reward exploit: MDP 9 32.42959308069246 20
average reward exploit: MDP 9 33.481759532472026 20
average reward exploit: MDP 9 33.41089744340023 20
average reward exploit: MDP 9 33.730333182452895 20
average reward exploit: MDP 9 33.81382350476576 20
average reward exploit: MDP 9 33.12364724991622 20
average reward exploit: MDP 9 34.19589678819613 20
evaluating on new MDPs
gravity:  [  0.          0.        -24.9432616]
average reward exploit 495.4917118549235 400
epoch number:  81
average reward explore: 698.111974824366
average reward exploit: MDP 0 26.97465865112316 20
average reward exploit: MDP 0 27.7292353916592 20
average reward exploit: MDP 0 28.434158417211115 20
average reward exploit: MDP 0 29.49006781847977 20
average reward

average reward exploit: MDP 3 35.26733123121928 20
average reward exploit: MDP 3 36.23504763725587 20
average reward exploit: MDP 3 39.00123971104057 20
average reward exploit: MDP 3 35.24313942455955 20
average reward exploit: MDP 3 36.98171191680188 20
average reward exploit: MDP 3 37.5660465086274 20
average reward exploit: MDP 3 38.60034160311937 20
average reward exploit: MDP 3 39.074545328334764 20
average reward exploit: MDP 3 37.62159934445111 20
average reward explore: 661.3353318971048
average reward exploit: MDP 4 36.7761742713716 20
average reward exploit: MDP 4 35.320362079172085 20
average reward exploit: MDP 4 36.558824404284444 20
average reward exploit: MDP 4 35.262913168857025 20
average reward exploit: MDP 4 37.74862877455413 20
average reward exploit: MDP 4 37.29494375960586 20
average reward exploit: MDP 4 36.92757710102992 20
average reward exploit: MDP 4 36.39154542291804 20
average reward exploit: MDP 4 36.38910191015379 20
average reward exploit: MDP 4 37.09995

average reward exploit: MDP 7 35.605314426346325 20
average reward exploit: MDP 7 36.253986513936205 20
average reward exploit: MDP 7 36.60600632896799 20
average reward explore: 512.5000541748444
average reward exploit: MDP 8 35.5459071540176 20
average reward exploit: MDP 8 36.92031445346015 20
average reward exploit: MDP 8 36.11878991647765 20
average reward exploit: MDP 8 36.217523864533895 20
average reward exploit: MDP 8 37.10101457048161 20
average reward exploit: MDP 8 37.382440682227454 20
average reward exploit: MDP 8 37.542451627179965 20
average reward exploit: MDP 8 38.31360822416643 20
average reward exploit: MDP 8 38.099292221714315 20
average reward exploit: MDP 8 39.835015930367796 20
average reward explore: 603.0063755630965
average reward exploit: MDP 9 43.38515897607233 20
average reward exploit: MDP 9 41.658651270443116 20
average reward exploit: MDP 9 44.53475344574626 20
average reward exploit: MDP 9 46.189629007279514 20
average reward exploit: MDP 9 46.95432696

average reward exploit: MDP 2 113.2411187896067 20
average reward exploit: MDP 2 84.98834586488336 20
average reward exploit: MDP 2 83.36144330911146 20
average reward exploit: MDP 2 101.1714247431327 20
average reward exploit: MDP 2 104.11910242269616 20
average reward exploit: MDP 2 89.64933304435064 20
average reward exploit: MDP 2 98.92681000219831 20
average reward exploit: MDP 2 112.16437354442829 20
average reward exploit: MDP 2 101.62784421850317 20
average reward exploit: MDP 2 102.87186174215648 20
average reward explore: 318.836877511507
average reward exploit: MDP 3 72.82025403198534 20
average reward exploit: MDP 3 73.18437655904819 20
average reward exploit: MDP 3 76.60705720881786 20
average reward exploit: MDP 3 78.53066250140725 20
average reward exploit: MDP 3 73.33187440226837 20
average reward exploit: MDP 3 75.54743788162185 20
average reward exploit: MDP 3 72.81062520172694 20
average reward exploit: MDP 3 68.89977358036137 20
average reward exploit: MDP 3 67.8539

KeyboardInterrupt: 

In [ ]:
a.lr = 3e-3
num_epochs = 50
for epoch in range(num_epochs):
        print("epoch number: ", epoch)
        a.train_step()

In [123]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,1)

In [124]:
print("average reward explore:", np.mean(explore_rewards), len(explore_rewards))
M = a.stack_trajectories(explore_paths)

average reward explore: 151.57677928797034 400


In [125]:
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })

In [126]:
Z = np.reshape(Z,[1,len(Z)])

In [127]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)

In [128]:
print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit 18.577710931432883 400


In [122]:
a.num_traj = 400

In [83]:
for i in range(10):
    maxrev2 = 0
    maxrev1 = 0
    traj = np.random.rand()*(i+1)
    
    for i in range(4):
        explore_paths, explore_rewards = a.sample_paths_explore(a.env,traj)
        #print("average reward explore:", np.mean(explore_rewards))
        M = a.stack_trajectories(explore_paths)
        Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
        Z = np.reshape(Z,[1,len(Z)])
        exploit_paths, explore_rewards = a.sample_paths_exploit(a.env,Z,traj)
        if(np.mean(explore_rewards) > maxrev1):
            maxrev1 = np.mean(explore_rewards) / num_traj
        if(np.mean(exploit_rewards)>maxrev2):
            maxrev2 = np.mean(exploit_rewards) / num_traj
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore", maxrev1)
    print("average reward exploit", maxrev2)
    

gravity: [ 0.         0.        -5.4325712]
average reward explore 3.5619071147766848
average reward exploit 2.7301699712255383


KeyboardInterrupt: 

In [90]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,5)
observations_explore = np.concatenate([path["observation"] for path in explore_paths])
new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
actions_explore = np.concatenate([path["action"] for path in explore_paths])
rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
returns_explore = a.get_returns(explore_paths, explore = True)

In [91]:
M = np.array(a.stack_trajectories(explore_paths))
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
Z = np.reshape(Z,[1,len(Z)])

In [92]:
 print("average reward explore: MDP", np.sum(explore_rewards)/num_traj, len(explore_rewards))

average reward explore: MDP 20488.85105031593 800


In [93]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,5)

In [94]:
print("average reward exploit: MDP", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit: MDP 644.4290518284578 400


In [ ]:
def randomize_pendulum(env):
    m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])
    l = config['pendulum_len_min'] + np.random.rand()*(config['pendulum_len_max'] - config['pendulum_len_min'])
    env.m = m
    env.l = l

In [ ]:
def randomize_hopper(env):
    ts = config['torso_min'] + np.random.rand()*(config['torso_max'] - config['torso_min'])
    f = config['friction_min'] + np.random.rand()*(config['friction_max'] - config['friction_min'])
    
    env.friction = f
    env.torso_size = ts
    env.apply_env_modifications()

In [ ]:
import yaml
cfg_filename = 'hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
env = Randomizer(gym.make('Hopper-v2'), randomize_hopper)
env.unwrapped.__dict__.keys()
env.reset()

In [ ]:
m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])

In [ ]:
config

In [ ]:
env_id = 'Hopper-v2'
e = gym.make(env_id)

In [ ]:
e.unwrapped.__dict__.keys()

In [ ]:
e.unwrapped.apply_env_modifications()

In [7]:
a.sim.model.opt